In [3]:
!conda install -c conda-forge folium=0.5.0 --yes 
!pip install shapely

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.19 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00 166.54 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  24.89 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  30.65 MB/s
Requirement not upgraded as not directly required: shapely in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
!wget -q -O 'london_data.json' http://data.insideairbnb.com/united-kingdom/england/london/2018-08-08/visualisations/neighbourhoods.geojson
!wget -q -O 'san_francisco_data.json' http://data.insideairbnb.com/united-states/ca/san-francisco/2018-08-06/visualisations/neighbourhoods.geojson
!wget -q -O 'sydney_data.json' http://data.insideairbnb.com/australia/nsw/sydney/2018-08-08/visualisations/neighbourhoods.geojson
!wget -q -O 'amsterdam_data.json' http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2018-08-07/visualisations/neighbourhoods.geojson
!wget -q -O 'paris_data.json' http://data.insideairbnb.com/france/ile-de-france/paris/2018-08-13/visualisations/neighbourhoods.geojson

In [27]:
cities = ['Paris', 'London', 'Sydney', 'San Francisco', 'Amsterdam']
geo_data ={}

with open('london_data.json') as json_data:
    geo_data['London'] = json.load(json_data)
    
with open('san_francisco_data.json') as json_data:
    geo_data['San Francisco'] = json.load(json_data)    
    
with open('sydney_data.json') as json_data:
    geo_data['Sydney'] = json.load(json_data)
    
with open('amsterdam_data.json') as json_data:
    geo_data['Amsterdam'] = json.load(json_data)      
    
with open('paris_data.json') as json_data:
    geo_data['Paris'] = json.load(json_data)           
    


(2.3062042884323084, 48.8870345)

In [47]:
from shapely.geometry import shape

column_names = ['City', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

for city in cities:
    for data in geo_data[city]['features']:
        neighborhood_name = data['properties']['neighbourhood']
        
        neighborhood_latlon = shape(data['geometry']).representative_point()._get_coords()[0]
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
        
        neighborhoods = neighborhoods.append({'City':city,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,City,Neighborhood,Latitude,Longitude
0,Paris,Batignolles-Monceau,48.887034,2.306204
1,Paris,Palais-Bourbon,48.854715,2.312969
2,Paris,Buttes-Chaumont,48.887149,2.383096
3,Paris,Opéra,48.877029,2.337891
4,Paris,Entrepôt,48.875907,2.360472


In [48]:
city = 'Paris'

df = neighborhoods[neighborhoods['City'] == city]

latitude = df.iloc[0]['Latitude']
longitude = df.iloc[0]['Longitude']

kclusters = 3
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

mapc = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, city, neighborhood in zip(df['Latitude'], df['Longitude'], df['City'], df['Neighborhood']):

    label = folium.Popup(neighborhood, parse_html=True)
    
    cluster = 1
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapc)
    
mapc

,neighbourhood,city,country
0,Central & Western,Hong Kong,China
1,Eastern,Hong Kong,China
2,Islands,Hong Kong,China
3,Kowloon City,Hong Kong,China
4,Kwai Tsing,Hong Kong,China
5,Kwun Tong,Hong Kong,China
6,North,Hong Kong,China
7,Sai Kung,Hong Kong,China
8,Sham Shui Po,Hong Kong,China
9,Sha Tin,Hong Kong,China


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [71]:
pd_hong_kong

,neighbourhood,city,country
0,Central & Western,Hong Kong,China
1,Eastern,Hong Kong,China
2,Islands,Hong Kong,China
3,Kowloon City,Hong Kong,China
4,Kwai Tsing,Hong Kong,China
5,Kwun Tong,Hong Kong,China
6,North,Hong Kong,China
7,Sai Kung,Hong Kong,China
8,Sham Shui Po,Hong Kong,China
9,Sha Tin,Hong Kong,China


In [69]:
import geocoder

def get_geo(neighbourhood, city, country):

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        print('{}, {}, {}'.format(neighbourhood, city, country))
        g = geocoder.google('{}, {}, {}'.format(neighbourhood, city, country))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return latitude, longitude


In [70]:
pd_hong_kong['LongLat'] = pd_hong_kong.apply(lambda row: get_geo(row['neighbourhood'],row['city'],row['country']), axis=1)
pd_hong_kong

Central & Western, Hong Kong, China
Eastern, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Islands, Hong Kong, China
Is

KeyboardInterrupt: 